<a href="https://colab.research.google.com/github/Lou1108/DeepLearning/blob/main/Assignment2%20/meg_prediction_sofia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



What we need to do
*   do the pre processing seperately and save the data
*   check and try different models to see if there's one working better with the data
* grid search on the number of the neurons in the layers



### Assignment 2

# Imports and Variables


In [107]:
import os
import glob
import h5py
import numpy as np
from collections import Counter

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.signal import butter, filtfilt, decimate

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input,
    Conv1D,
    Conv2D,
    DepthwiseConv2D,
    SeparableConv2D,
    MaxPooling1D,
    AveragePooling2D,
    LSTM,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
    Activation,
)


In [87]:
TRAIN_PATH = "/content/drive/MyDrive/Deep Learning/Final Project data/Intra/train"
TEST_PATH = "/content/drive/MyDrive/Deep Learning/Final Project data/Intra/test"


# data specific
NUM_CHANNELS = 248
NUM_CLASSES = 4
LABEL_MAP = {'rest':0, 'task_motor':1, 'task_story_math':2, 'task_working_memory':3}
NUM_CLASSES = len(LABEL_MAP)
orig_fs=2034
target_fs=250
DOWNSAMPLE_FACTOR =  int(orig_fs / target_fs)

# Model specific
NUM_EPOCHS = 1
NORMALIZATION_METHOD = "minmax"  # Choose: "minmax", "zscore", or "perchannel"

In [64]:
def load_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        # Extractin the label
        filename = file_path.split('/')[-1]

        #handling the different task naming conventions
        if 'rest' in filename:
            labels.append(LABEL_MAP['rest'])
        elif 'motor' in filename:
            labels.append(LABEL_MAP['task_motor'])
        elif 'story' in filename or 'math' in filename:
             labels.append(LABEL_MAP['task_story_math'])
        elif 'working' in filename or 'memory' in filename:
            labels.append(LABEL_MAP['task_working_memory'])
        else:
            # iff a file doesn't match
            print(f"Could not determine task for file: {filename}")
            continue

        with h5py.File(file_path, 'r') as f:
            # Instead of guessing the dataset name, we get the first key from the file
            # This is robust because we know there is only one dataset per file[cite: 10].
            dataset_name = list(f.keys())[0]
            matrix = f[dataset_name][()]
            data.append(matrix)

    #convert to numpy arrays
    return np.array(data), np.array(labels)

In [67]:
train_files = glob.glob(f"{TRAIN_PATH}/*.h5")
test_files = glob.glob(f"{TEST_PATH}/*.h5")

X_train, y_train = load_data(train_files)
X_test, y_test = load_data(test_files)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Uniqe labels: {np.unique(y_train)}")
print(f"Number of training samples: {len(X_train)}")

Shape of X_train: (32, 248, 35624)
Shape of y_train: (32,)
Shape of X_test: (8, 248, 35624)
Uniqe labels: [0 1 2 3]
Number of training samples: 32


# Load and Preprocess Data
Apply a lowpass filter for downsampling the frequency

In [79]:
# lowpass filter ---> check it
# def bandpass_filter(data, lowcut=1.0, highcut=150.0, fs=2034, order=5):
#     nyq = 0.5 * fs
#     low = lowcut / nyq
#     high = highcut / nyq
#     b, a = butter(order, [low, high], btype='band')
#     return filtfilt(b, a, data, axis=-1)

Normalization functions

In [80]:
def min_max_scale_sample(data):
    scaler = MinMaxScaler()
    return scaler.fit_transform(data.reshape(-1, 1)).reshape(data.shape)

def z_score_normalize(data):
    mean = data.mean(axis=-1, keepdims=True)
    std = data.std(axis=-1, keepdims=True)
    return (data - mean) / (std + 1e-8)

def time_norm(data):
    n_samples, n_channels, n_timesteps = data.shape
    reshaped_data = data.reshape(n_samples * n_channels, n_timesteps)

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(reshaped_data)

    # Reshape back to the original shape
    return scaled_data.reshape(n_samples, n_channels, n_timesteps)

In [82]:
def normalization(data):
    if NORMALIZATION_METHOD == "minmax":
        data = min_max_scale_sample(data)
    elif NORMALIZATION_METHOD == "zscore":
        data = z_score_normalize(data)
    elif NORMALIZATION_METHOD == "time":
        data = time_norm(data)
    return data

In [83]:
N_TIMESTEPS = X_train.shape[2]
X_train_ds=decimate(X_train, DOWNSAMPLE_FACTOR, axis=-1, ftype='fir', zero_phase=True)
X_test_ds=decimate(X_test, DOWNSAMPLE_FACTOR, axis=-1, ftype='fir', zero_phase=True)
N_TIMESTEPS_DS = X_train_ds.shape[2]

print(f"Original number of time steps: {N_TIMESTEPS}")
print(f"Downsampled number of time steps: {N_TIMESTEPS_DS}")

Original number of time steps: 35624
Downsampled number of time steps: 4453


In [84]:
X_train_norm = normalization(X_train_ds)
X_test_norm = normalization(X_test_ds)


Normalizing data...


In [85]:
#DL models in Keras often expect the channel dimension last
#reshaping from (samples, channels, timesteps) to (samples, timesteps, channels)
X_train_final = np.transpose(X_train_norm, (0, 2, 1))
X_test_final = np.transpose(X_test_norm, (0, 2, 1))

print(f"Final shape of training data for the model: {X_train_final.shape}")

Final shape of training data for the model: (32, 4453, 248)


In [102]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_final,
    y_train,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=y_train      # ← ensures all 4 classes are in both sets
)

print(f"Train samples: {len(X_tr)}, Val samples: {len(X_val)}")

Train samples: 25, Val samples: 7


# Define Models

###CNN models

CNN sequential

In [103]:
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),

        # just one light conv block
        Conv1D(32, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(2),

        Flatten(),

        # very small dense head
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax'),
    ])
    return model

In [106]:
INPUT_SHAPE = (N_TIMESTEPS_DS, NUM_CHANNELS)

cnn_model = build_cnn_model(INPUT_SHAPE, NUM_CLASSES)
cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy', # Use sparse CE because our labels are integers
    metrics=['accuracy']
)
cnn_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 4453, 32)       │        23,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 4453, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 2226, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 71232)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │     2,279,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,303,556 (8.79 MB)

 Trainable params: 2,303,492 (8.79 MB)

 Non-trainable params: 64 (256.00 B)

CNN LSTM

In [110]:
def build_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),

        # Convolutional layers to extract features
        Conv1D(filters=64, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # LSTM layer to model temporal sequences of the extracted features
        LSTM(128, return_sequences=False), # return_sequences=False  it's the last recurrent layer

        # Dense layers for classification
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

In [111]:
cnn_lstm_model = build_cnn_lstm_model(INPUT_SHAPE, NUM_CLASSES)

cnn_lstm_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cnn_lstm_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_7 (Conv1D)               │ (None, 4453, 64)       │       158,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 4453, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_7 (MaxPooling1D)  │ (None, 1113, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 1113, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 1113, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 278, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 390,212 (1.49 MB)

 Trainable params: 389,828 (1.49 MB)

 Non-trainable params: 384 (1.50 KB)

EEGNet

In [115]:
def build_eegnet_model(num_classes, channels, timesteps, dropout_rate=0.5):
    input_layer = Input(shape=(channels, timesteps, 1))

    # temporal convolution
    block1 = Conv2D(16, (1, 64), padding='same', use_bias=False)(input_layer)
    block1 = BatchNormalization()(block1)

    #Depthwise spatial convolution
    block1 = DepthwiseConv2D((channels, 1), use_bias=False, depth_multiplier=2, depthwise_constraint=tf.keras.constraints.max_norm(1.))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropout_rate)(block1)

    # separable convolution
    block2 = SeparableConv2D(32, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropout_rate)(block2)

    # classification head
    flatten_layer = Flatten()(block2)
    dense_layer = Dense(num_classes, kernel_constraint=tf.keras.constraints.max_norm(0.25))(flatten_layer)
    output_layer = Activation('softmax')(dense_layer)

    return Model(inputs=input_layer, outputs=output_layer)

In [125]:
X_train_eegnet = np.transpose(X_train_norm, (0, 2, 1))
X_train_eegnet = X_train_norm[..., np.newaxis]
X_test_eegnet = np.transpose(X_test_norm, (0, 2, 1))
X_test_eegnet = X_test_norm[..., np.newaxis]
print(f"Shape of data for EEGNet: {X_train_eegnet.shape}")

Shape of data for EEGNet: (32, 248, 4453, 1)


In [126]:
eegnet_model = build_eegnet_model(
    num_classes=NUM_CLASSES,
    channels=NUM_CHANNELS,
    timesteps=N_TIMESTEPS_DS
)

eegnet_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

eegnet_model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 248, 4453, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 248, 4453, 16)  │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 248, 4453, 16)  │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 1, 4453, 32)    │         7,936 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 1, 4453, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 1, 4453, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 1, 1113, 32)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 1, 1113, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 1, 1113, 32)    │         1,536 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 1, 1113, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1, 1113, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 1, 139, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 1, 139, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 4448)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │        17,796 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,612 (111.77 KB)

 Trainable params: 28,452 (111.14 KB)

 Non-trainable params: 160 (640.00 B)

### Training and evaluation of CNNs

* CNN Sequential

In [112]:
callbacks = [
    EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6
    ),
]

history = cnn_model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=4,
    callbacks=callbacks,
    verbose=2
)

# Evaluate on test Sets
loss_cnn, acc_cnn = cnn_model.evaluate(X_test_final, y_test, verbose=0)
print(f"accuracy on test set: {acc_cnn * 100:.2f}%")

Epoch 1/10
7/7 - 1s - 181ms/step - accuracy: 0.7600 - loss: 0.5310 - val_accuracy: 0.2857 - val_loss: 3.0461 - learning_rate: 5.0000e-05
Epoch 2/10
7/7 - 1s - 173ms/step - accuracy: 0.8000 - loss: 0.6750 - val_accuracy: 0.2857 - val_loss: 3.0058 - learning_rate: 5.0000e-05
Epoch 3/10
7/7 - 1s - 173ms/step - accuracy: 0.8800 - loss: 0.6362 - val_accuracy: 0.2857 - val_loss: 5.7636 - learning_rate: 5.0000e-05
Epoch 4/10
7/7 - 1s - 131ms/step - accuracy: 0.8800 - loss: 0.3170 - val_accuracy: 0.2857 - val_loss: 7.9567 - learning_rate: 5.0000e-05
Epoch 5/10
7/7 - 1s - 168ms/step - accuracy: 0.8000 - loss: 0.3678 - val_accuracy: 0.2857 - val_loss: 8.9843 - learning_rate: 5.0000e-05
Epoch 6/10
7/7 - 1s - 107ms/step - accuracy: 0.8800 - loss: 0.3184 - val_accuracy: 0.2857 - val_loss: 8.4658 - learning_rate: 2.5000e-05
Epoch 7/10
7/7 - 1s - 179ms/step - accuracy: 0.8800 - loss: 0.3609 - val_accuracy: 0.2857 - val_loss: 7.4583 - learning_rate: 2.5000e-05
accuracy on test set: 25.00%


* LSTM

In [114]:
history_cnn_lstm = cnn_lstm_model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=4,
    callbacks=callbacks,
    verbose=2
)

# Evaluate on test sets
loss_hybrid, acc_hybrid = cnn_lstm_model.evaluate(X_test_final, y_test, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 1: {acc_hybrid * 100:.2f}%")

Epoch 1/10
7/7 - 3s - 495ms/step - accuracy: 0.8800 - loss: 0.4858 - val_accuracy: 0.2857 - val_loss: 1.3798 - learning_rate: 5.0000e-04
Epoch 2/10
7/7 - 7s - 938ms/step - accuracy: 0.9200 - loss: 0.3130 - val_accuracy: 0.4286 - val_loss: 1.3316 - learning_rate: 5.0000e-04
Epoch 3/10
7/7 - 4s - 517ms/step - accuracy: 0.9200 - loss: 0.2937 - val_accuracy: 0.2857 - val_loss: 1.3911 - learning_rate: 5.0000e-04
Epoch 4/10
7/7 - 5s - 730ms/step - accuracy: 0.9600 - loss: 0.2407 - val_accuracy: 0.2857 - val_loss: 1.4129 - learning_rate: 5.0000e-04
Epoch 5/10
7/7 - 6s - 803ms/step - accuracy: 0.9200 - loss: 0.3759 - val_accuracy: 0.2857 - val_loss: 1.4649 - learning_rate: 5.0000e-04
Epoch 6/10
7/7 - 3s - 482ms/step - accuracy: 0.9200 - loss: 0.4065 - val_accuracy: 0.2857 - val_loss: 1.4560 - learning_rate: 2.5000e-04
Epoch 7/10
7/7 - 3s - 487ms/step - accuracy: 0.9600 - loss: 0.2838 - val_accuracy: 0.2857 - val_loss: 1.5013 - learning_rate: 2.5000e-04
Hybrid Model Accuracy on Test Set 1: 50.0

* EEGNet

In [127]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history_eegnet = eegnet_model.fit(
    X_train_eegnet,
    y_train,
    epochs=20,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping]
)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 97s 29s/step - accuracy: 0.3917 - loss: 1.3681 - val_accuracy: 0.0000e+00 - val_loss: 1.3974
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 128s 30s/step - accuracy: 0.7358 - loss: 0.8278 - val_accuracy: 0.0000e+00 - val_loss: 1.4062
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 86s 30s/step - accuracy: 0.8308 - loss: 0.4270 - val_accuracy: 0.4286 - val_loss: 1.4052
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 143s 33s/step - accuracy: 0.9258 - loss: 0.2064 - val_accuracy: 0.4286 - val_loss: 1.4096
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 137s 32s/step - accuracy: 1.0000 - loss: 0.1107 - val_accuracy: 0.4286 - val_loss: 1.4097
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 78s 29s/step - accuracy: 1.0000 - loss: 0.0494 - val_accuracy: 0.4286 - val_loss: 1.4189


In [129]:
loss_eegnet, acc_eegnet = eegnet_model.evaluate(X_test_eegnet, y_test, verbose=0)
print(f"EEGNet Model Accuracy on Test Set: {acc_eegnet * 100:.2f}%")

EEGNet Model Accuracy on Test Set: 25.00%
